In [1]:
!pip3 install opencv-python

# apt-get update && apt-get install ffmpeg libsm6 libxext6 -y


[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import os
import cv2
import pickle
import numpy as np
import json

In [2]:
root_path = "/home/rafay/Documents/rafay/MyFolder_other_stuff/WSL_and_Stuff/other/open_source/3D_RetinaNet_ObjectDetection"

In [3]:
anno_file = os.path.join(root_path, "road/road_trainval_v1.0.json")

with open(anno_file, "r") as fff:
    final_annots = json.load(fff)

In [4]:
# Define class lists
agent_labels = final_annots["agent_labels"]
action_labels = final_annots["action_labels"]
loc_labels = final_annots["loc_labels"]
duplex_labels = final_annots["duplex_labels"]
triplet_labels = final_annots["triplet_labels"]
del final_annots

In [7]:
def get_pred_class_labels(detection_):
    """
    This function extracts the predicted class labels from a detection array.

    Parameters:
    detection_ (numpy.ndarray): A 1D numpy array representing a detection. The array contains
        probabilities for different classes. The first 5 elements are not used for class prediction.
        The elements from index 5 to 14 represent probabilities for agent labels.
        The elements from index 15 to 33 represent probabilities for action labels.
        The elements from index 34 to 45 represent probabilities for location labels.
        The elements from index 46 to 84 represent probabilities for duplex labels.
        The elements from index 85 onwards represent probabilities for triplet labels.

    Returns:
    tuple: A tuple containing the predicted class labels for agent, action, location, duplex, and triplet.
    """
    agent_label_ = agent_labels[np.argmax(detection_[5:15])]
    action_label_ = action_labels[np.argmax(detection_[15:34])]
    loc_label_ = loc_labels[np.argmax(detection_[34:46])]
    duplex_label_ = duplex_labels[np.argmax(detection_[46:85])]
    triplet_label_ = triplet_labels[np.argmax(detection_[85:])]
    return agent_label_, action_label_, loc_label_, duplex_label_, triplet_label_

In [8]:
def annotate_frames(det_save_dir, video_frames_dir, output_dir):
    """
    Annotate video frames with object detections and save the annotated frames.

    This function processes detection files for each video, loads corresponding frames,
    draws bounding boxes and labels for detected objects, and saves the annotated frames.

    Parameters:
    det_save_dir (str): Directory containing detection files for each video.
    video_frames_dir (str): Directory containing original video frames.
    output_dir (str): Directory where annotated frames will be saved.

    Returns:
    None. The function saves annotated frames to the specified output directory.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for video_name in os.listdir(det_save_dir):
        video_det_path = os.path.join(det_save_dir, video_name)
        video_frame_path = os.path.join(video_frames_dir, video_name)
        output_video_dir = os.path.join(output_dir, video_name)

        if not os.path.exists(output_video_dir):
            os.makedirs(output_video_dir)

        detection_files = sorted(os.listdir(video_det_path))  # Ensure file order
        for det_file in detection_files:
            frame_index = int(det_file.split('.')[0])  # Extract frame number
            det_path = os.path.join(video_det_path, det_file)
            frame_path = os.path.join(video_frame_path, f"{frame_index:05d}.jpg")  # Assuming frames are named numerically

            if not os.path.exists(frame_path):
                print(f"Frame {frame_path} not found, skipping.")
                continue

            # Load the frame
            frame = cv2.imread(frame_path)

            # Load detections
            with open(det_path, "rb") as f:
                detection_data = pickle.load(f)

            # Annotate object detections
            for det in detection_data["main"]:
                bbox = det[:4]  # x1, y1, x2, y2
                score = det[4]
                x1, y1, x2, y2 = map(int, bbox)
                x1, y1, x2, y2 = int(x1*1.9), int(y1*1.875), int(x2*1.9), int(y2*1.875)
                agent_label_, action_label_, loc_label_, duplex_label_, triplet_label_ = get_pred_class_labels(det)
                COLORS = (np.random.randint(0,255), np.random.randint(0,255), np.random.randint(0,255))
                if score>=0.5:
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color=COLORS, thickness=2)  # random unique list(COLORS)
                    cv2.putText(
                        frame, " ".join(triplet_label_, ), (x1, y1 - 15),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color=COLORS, thickness=1
                    )

            # Save annotated frame
            output_frame_path = os.path.join(output_video_dir, f"{frame_index:05d}.jpg")
            cv2.imwrite(output_frame_path, frame)
            print(f"Annotated frame saved to {output_frame_path}.")

In [10]:
# Paths
root_path
det_save_dir = os.path.join(root_path, "road/temp_ann_dir/detections")
video_frames_dir = os.path.join(root_path, "road/temp_ann_dir/frames")
output_dir = os.path.join(root_path, "road/temp_ann_dir/annotated")

In [ ]:
# Annotate frames
annotate_frames(det_save_dir, video_frames_dir, output_dir)